<a id=march></a>
## Export Tweets to Gephi CSVs

In [ ]:
import pickle
import csv

data = pickle.load(open("tweets.pkl", "rb"))

print(len(data))

In [ ]:
sn_list = [t.user.screen_name for t in data]

for t in data:
    for sn in t.entities['user_mentions']:
        sn_list.append(sn['screen_name'])
        
sn_list = list(set(sn_list))

In [ ]:
sn_dict = {}

idn = 1
for n in sn_list:
    sn_dict[n] = {'id': idn,
                  'mentions': []
                 }
    idn += 1

In [ ]:
for t in data:
    for sn in t.entities["user_mentions"]:
        sn_dict[t.user.screen_name]['mentions'].append(sn_dict[sn["screen_name"]]['id'])

In [ ]:
id_rows = [['Nodes', 'Id']]

for k in sn_dict.keys():
    id_rows.append([k, sn_dict[k]['id']])

with open("id_sns.csv", "w") as fp:
    wr = csv.writer(fp, dialect='excel')
    wr.writerows(id_rows)

In [ ]:
mention_rows = [['Source', 'Target']]

for k in sn_dict.keys():
    if len(sn_dict[k]['mentions']) > 0:
        if len(list(set(sn_dict[k]['mentions']))) == 1:
            mention_rows.append([sn_dict[k]['id']] + list(set(sn_dict[k]['mentions'])))
        else:
            for m in list(set(sn_dict[k]['mentions'])):
                mention_rows.append([sn_dict[k]['id']] + [m])

with open("id_mentions.csv", "w") as fp:
    wr = csv.writer(fp, dialect='excel')
    wr.writerows(mention_rows)